# Decisiones de Particionamiento en Azure Databricks

### 1. Introducción
El particionamiento de datos es una técnica clave en sistemas distribuidos que permite dividir grandes conjuntos de datos en partes más pequeñas para mejorar el rendimiento de las consultas y la eficiencia de los recursos. Sin embargo, particionar innecesariamente o en exceso puede causar un impacto negativo en el rendimiento. Aquí vamos a evaluar cuándo particionar y cuándo no particionar en función de factores como el tamaño del dataset, las consultas que realizamos, y la cardinalidad de las columnas.

### 2. Tamaño del Dataset y Decisión de Particionamiento
**Dataset pequeño (menos de 1GB o 500,000 filas)**
Regla general: Si tu dataset es pequeño, no tiene sentido particionarlo, ya que el overhead que se genera al gestionar particiones podría ser mayor que los beneficios.

In [0]:
# Cargar un dataset pequeño
df_small = spark.read.csv("/mnt/datalake/small_dataset.csv", header=True, inferSchema=True)

# Contar registros
df_small.count()

# Ejecutar consulta sin particionar
import time
start_time = time.time()
df_small.groupBy("product_category").sum("sales").show()
print("--- %s seconds ---" % (time.time() - start_time))


Resultado esperado: Como el dataset es pequeño, no deberías ver una mejora significativa al particionar, ya que el tamaño es manejable en un solo archivo.

### Dataset grande (más de 1GB o millones de filas)
**Regla general:** Si el dataset es grande (más de 1GB o millones de filas), el particionamiento es crucial para mejorar el rendimiento.

In [0]:
# Cargar un dataset grande
df_large = spark.read.csv("/mnt/datalake/large_dataset.csv", header=True, inferSchema=True)

# Evaluar el número de registros
df_large.count()

# Particionar los datos por 'region' y 'year' y guardarlos en Delta Lake
df_large.repartition("region", "year").write.partitionBy("region", "year").mode("overwrite").format("delta").save("/mnt/datalake/partitioned_data/")


Resultado esperado: Las consultas sobre el dataset particionado deberían mostrar mejoras en el rendimiento, ya que solo se leerán las particiones necesarias en lugar de todo el dataset.

### 3. Patrón de Acceso a los Datos
**Escaneo completo (sin particionar)**
Si las consultas escanean todo el dataset sin filtros por columnas, el particionamiento puede ser innecesario y podría introducir sobrecarga adicional.

In [0]:
# Ejecutar una consulta que escanee todo el dataset
start_time = time.time()
df_large.groupBy("product_category").sum("sales").show()
print("--- %s seconds ---" % (time.time() - start_time))

**Resultado esperado:** Si las consultas no filtran por columnas, el particionamiento no ofrece ventajas claras, ya que las particiones no se están aprovechando.

### Filtrado por columnas (particionar sí es útil)
Si las consultas frecuentes filtran por columnas específicas (como region o year), el particionamiento mejora significativamente el rendimiento.

In [0]:
# Filtrar por columna particionada (region) y medir el tiempo
df_partitioned = spark.read.format("delta").load("/mnt/datalake/partitioned_data/")
start_time = time.time()
df_partitioned.filter("region = 'North' AND year = 2021").groupBy("product_category").sum("sales").show()
print("--- %s seconds ---" % (time.time() - start_time))


**Resultado esperado:** Al particionar por las columnas filtradas, la consulta será mucho más rápida que sin particionar, ya que Spark solo leerá las particiones relevantes.

### 4. Cardinalidad de las Columnas y Impacto del Particionamiento
**Alta cardinalidad (particionar sí es útil)**
Particionar por columnas con muchos valores únicos (como region, year) distribuye los datos de manera más uniforme y mejora el paralelismo.

In [0]:
# Particionar por 'region' y 'year' (alta cardinalidad)
df_large.repartition("region", "year").write.partitionBy("region", "year").mode("overwrite").parquet("/mnt/datalake/partitioned_by_region_year/")


**Baja cardinalidad (evitar particionamiento)**
Si particionas por columnas con pocos valores únicos (baja cardinalidad, como gender o boolean), las particiones no aportan beneficios significativos y pueden incluso reducir el paralelismo.

In [0]:
# Particionar por columna de baja cardinalidad (como gender)
df_large.repartition("gender").write.partitionBy("gender").mode("overwrite").parquet("/mnt/datalake/partitioned_by_gender/")


**Resultado esperado:** Particionar por columnas con baja cardinalidad puede resultar en particiones grandes, lo que reduce el paralelismo y no mejora el rendimiento.

### 5. Monitoreo del Impacto del Particionamiento
**Spark UI:** Para cada consulta, utiliza la **Spark UI** para analizar cómo las particiones afectan el tiempo de ejecución, la cantidad de "shuffle", y el uso de memoria.

Navega a la **Spark UI** y examina la sección de "Jobs" para ver cuántas particiones fueron procesadas y cuántos datos fueron "shuffleados" entre nodos.
**Ganglia Metrics:** Utiliza **Ganglia** para monitorear el uso de CPU y memoria. Si el uso de recursos no mejora después de particionar, revisa tu estrategia de particionamiento.

# Cuándo particionar los datos
### Grandes volúmenes de datos (mayores de 1GB o millones de filas)

**Razón:** Los datasets grandes tienden a requerir particiones para aprovechar el procesamiento distribuido en clústeres de Databricks y mejorar el rendimiento en consultas. Particionar permite que Spark procese datos en paralelo, distribuyendo las tareas entre nodos del clúster.

**Tamaño recomendado:** Generalmente, datasets que superen 1GB o aquellos con millones de filas se benefician de particionamiento, ya que leer y procesar todo el dataset sin particiones puede ser ineficiente.

**Ejemplo:** En un dataset de ventas con transacciones de varios años, particionar por year, region, o product_category ayuda a reducir el volumen de datos que Spark tiene que leer y procesar para consultas específicas.

**Mejoras esperadas:** Menor tiempo de ejecución para consultas que filtran por columnas particionadas y menor uso de recursos al leer solo las particiones necesarias.
### Consultas que filtran por columnas específicas

**Razón:** Si las consultas frecuentes filtran por valores específicos (por ejemplo, year, region), particionar por esas columnas permite que Spark lea solo las particiones relevantes, en lugar de escanear todo el dataset.

**Tamaño de referencia:** Si el dataset es mayor a 500MB y las consultas se basan en columnas particulares con altos valores de cardinalidad, como fechas o categorías de productos.

**Mejoras esperadas:** Reducción significativa en el tiempo de consulta y mejora en el uso de recursos del clúster.
### Optimización de almacenamiento en sistemas distribuidos

**Razón:** Particionar es importante en sistemas como Azure Data Lake o S3, ya que evita la creación de un único archivo masivo que es ineficiente de procesar. En lugar de tener un archivo gigante, se dividen los datos en archivos más pequeños organizados por particiones.

**Tamaño de referencia:** Datasets con varios GB o TB de datos.

**Mejoras esperadas:** Mejor eficiencia en operaciones de lectura y escritura, ya que los archivos están distribuidos y segmentados.


# Cuándo NO particionar los datos
### Datasets pequeños (menos de 1GB)

**Razón:** Para datasets pequeños (menos de 1GB), particionar no suele ser necesario porque el overhead de crear particiones y leer varios archivos puede ser mayor que los beneficios. Spark puede procesar fácilmente datasets pequeños sin particionar.

**Tamaño de referencia:** Datasets menores a 500MB o con menos de 100,000 filas.

**Impacto negativo:** Si particionas un dataset pequeño, puedes crear demasiados archivos pequeños, lo que ralentiza las consultas y aumenta el overhead de administración de archivos.

### Consultas que no filtran o escanean el dataset completo

**Razón:** Si la mayoría de las consultas en tu dataset escanean todo el dataset sin filtrar por columnas específicas (por ejemplo, en un análisis de datos masivos o agregaciones a nivel global), particionar puede no tener beneficios significativos. De hecho, puede generar una sobrecarga adicional, ya que Spark debe procesar particiones innecesarias.

**Tamaño de referencia:** Si las consultas escanean el dataset completo y no se filtran por columnas específicas, particionar puede generar más archivos pequeños y sobrecargar el sistema sin mejorar el rendimiento.
Impacto negativo: Aumento de la sobrecarga de administración de particiones y archivos, sin mejora en el rendimiento.

### Columnas con baja cardinalidad o pocos valores únicos

**Razón:** Si particionas un dataset basado en una columna con pocos valores únicos (baja cardinalidad), el beneficio del particionamiento es limitado porque cada partición será grande. Por ejemplo, particionar por gender o boolean con solo dos valores (male, female o true, false) puede ser innecesario.

**Impacto negativo:** Crear demasiadas particiones grandes (cada partición representando un valor de la columna) puede reducir el paralelismo y no ofrecer mejoras en el rendimiento.